In [1]:
# 진행경과 표기.
# !pip install geopy
import pandas as pd
import time
import csv
import os
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from concurrent.futures import ThreadPoolExecutor, as_completed

# Nominatim API 초기화
geolocator = Nominatim(user_agent="geo_converter")

# CSV 파일 로드
df = pd.read_csv("./converted_addresses_전체_데이터_주소_1차중복제거.csv", encoding="utf-8")

# "주소" 필드 선택 후 리스트 변환
addresses = df["주소"].dropna().tolist()

# 전체 주소 개수
total_count = len(addresses)
progress_count = 0  # 진행 개수 추적

# 결과 저장 CSV 파일 경로
output_csv = "converted_addresses_전체데이터_주소_좌표_변환.csv"

# CSV 파일이 없는 경우 헤더 추가
if not os.path.exists(output_csv):
    with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["주소", "위도", "경도"])  # 헤더 추가

# 주소 변환 함수 (예외 처리 + 1.5초 대기 + 진행 상태 업데이트)
def get_lat_lon(address):
    global progress_count  # 진행 개수 업데이트를 위해 전역 변수 사용

    try:
        location = geolocator.geocode(address, timeout=10)  # 10초 대기
        if location:
            lat, lon = location.latitude, location.longitude
        else:
            lat, lon = None, None
    except GeocoderTimedOut:
        print(f"⏳ 시간 초과: {address}, 다시 시도 중...")
        time.sleep(2)
        return get_lat_lon(address)  # 재시도

    # 변환 완료 후 CSV에 한 줄씩 추가 저장
    with open(output_csv, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([address, lat, lon])

    # 진행 개수 업데이트
    progress_count += 1
    progress_percent = (progress_count / total_count) * 100
    print(f"✅ 진행 상태: {progress_count}/{total_count} 완료 ({progress_percent:.2f}%)")

    time.sleep(1.5)  # 1.5초 대기 (속도 제한 방지)
    return address, lat, lon

# 멀티스레딩 실행 (최대 5개 동시 실행)
with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_address = {executor.submit(get_lat_lon, addr): addr for addr in addresses}

    for future in as_completed(future_to_address):
        try:
            future.result()  # 각 변환 작업 실행
        except Exception as e:
            print(f"⚠️ 오류 발생: {e}")

print("🎉 모든 주소 변환 완료! 변환된 데이터가 '한국전력공사_부산시 전기차 충전소 충전량_20220331.csv'에 실시간으로 저장되었습니다.")


✅ 진행 상태: 1/23 완료 (4.35%)
✅ 진행 상태: 2/23 완료 (8.70%)
✅ 진행 상태: 3/23 완료 (13.04%)
✅ 진행 상태: 4/23 완료 (17.39%)
✅ 진행 상태: 5/23 완료 (21.74%)
✅ 진행 상태: 6/23 완료 (26.09%)
✅ 진행 상태: 7/23 완료 (30.43%)
✅ 진행 상태: 8/23 완료 (34.78%)
✅ 진행 상태: 9/23 완료 (39.13%)
✅ 진행 상태: 10/23 완료 (43.48%)
✅ 진행 상태: 11/23 완료 (47.83%)
✅ 진행 상태: 12/23 완료 (52.17%)
✅ 진행 상태: 13/23 완료 (56.52%)
✅ 진행 상태: 14/23 완료 (60.87%)
✅ 진행 상태: 15/23 완료 (65.22%)
✅ 진행 상태: 16/23 완료 (69.57%)
✅ 진행 상태: 17/23 완료 (73.91%)
✅ 진행 상태: 18/23 완료 (78.26%)
✅ 진행 상태: 19/23 완료 (82.61%)
✅ 진행 상태: 20/23 완료 (86.96%)
✅ 진행 상태: 21/23 완료 (91.30%)
✅ 진행 상태: 22/23 완료 (95.65%)
✅ 진행 상태: 23/23 완료 (100.00%)
🎉 모든 주소 변환 완료! 변환된 데이터가 '한국전력공사_부산시 전기차 충전소 충전량_20220331.csv'에 실시간으로 저장되었습니다.
